In [1]:
import logging
import time
from datetime import datetime

import geohash
import numpy as np
import pandas as pd
import geopandas as gpd
import dask
import dask.dataframe as dd
import dask.config
from distributed import LocalCluster, Client

In [2]:
# dask.config.config['distributed']['comm']['timeouts']['connect'] = '300s'
# dask.config.config['distributed']['scheduler']['allowed-failures'] = 12
# dask.config.config['distributed']['worker']['memory'] = {'target': 0.3, 'spill': 0.5, 'pause': 0.7, 'terminate': 0.95}

In [ ]:
n_workers=2
cluster = LocalCluster(n_workers=n_workers, threads_per_worker=6, memory_limit=f'{20E9/n_workers}')#silence_logs=logging.ERROR)
client = Client(cluster)
client

In [4]:
df = dd.read_parquet('../data/contiguous_us_w_geohash.parquet', engine='pyarrow')
df.head()

,latitude,longitude,geohash
index,,,
38605,24.396309,-75.881459,dk64c30ycce4
38630,24.396317,-109.907775,9s3ffcpwg5p1
38638,24.396320,-81.164672,dhq6c11wy5dj
38639,24.396320,-77.226879,dk34f9jng79m
38689,24.396338,-75.171110,dk6dc3nnfnvw


In [5]:
%%time 
t0 = time.time()
df.set_index('geohash', shuffle='disk').to_parquet('../data/contiguous_us_geohash_sorted.parquet')
dt_hr = (time.time() - t0)/60/60

CPU times: user 20.7 s, sys: 2.27 s, total: 23 s
Wall time: 5min 16s


In [6]:
with open(f'{datetime.now()}_us_geohash_sort_time.csv', 'w') as f:
    f.write(f'dt_hr, {dt_hr}')